## Bitboard für Onitama



In [5]:
import numpy as np

#### Board

Ein *board* ist ein 5x5 numpy-Array mit den Werten:
```
0 - frei
1 - Schüler Spieler 1
2 - Meister Spieler 1
3 - Schüler Spieler 2
4 - Meister Spieler2

```
 

Funktionen zum Erstellen und Füllen eines boards:

In [33]:
def makeBoard():
    return np.zeros((5, 5), dtype = int)
    
def startBoard():
    board = makeBoard()
    board[4] = np.array([1,1,2,1,1])
    board[0] = np.array([3,3,4,3,3])
    return board

def showBoard(board):
    for i in range(5):
        print(i,'|',*board[i,:])
    print('    =========')
    print('    A B C D E')
    print()
    


Das Startboard

In [34]:
board = startBoard()
showBoard(board)
 

0 | 3 3 4 3 3
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 1 1 2 1 1
    A B C D E



In [114]:
def boardzahlen(board):
    ''' returns int : zahl, die im bin-format das board repräsentiert'''
    bs1, bs2, bs3, bs4 = '', '', '', ''
    for j in range(4, -1, -1):
        for i in range(0, 5):
            bs1 += ['0', '1'][int(board[i, j] == 1)]
            bs2 += ['0', '1'][int(board[i, j] == 2)]
            bs3 += ['0', '1'][int(board[i, j] == 3)]
            bs4 += ['0', '1'][int(board[i, j] == 4)]
    return int(bs1, 2), int(bs2, 2), int(bs3, 2), int(bs4, 2)

In [116]:
board = startBoard()
bs1, bs2, bs3, bs4 = boardzahlen(board) 
bs1, bs2, bs3, bs4

(1081377, 1024, 17302032, 16384)

Wir betrachten den Bitstring, der durch die beiden Bitzahlen repräsentiert werden.

In [117]:
def bitboard(bz):
    bs = bitstr(bz)
    i, j = 4, 0
    board = np.zeros((5, 5), dtype = int)
    for c in bs[::-1]:
        board[i,j] = int(c)
        i-=1
        if i == -1:
            j += 1
            i = 4 
    showBoard(board)
    
def bitstr(bz):
    bitstr = bin(bz)[2:]
    return bitstr.rjust(25,'0')

In [63]:
print(bitstr(bs1), bitstr(bs2), bitstr(bs3), bitstr(bs4))
bitboard(bs1)
bitboard(bs2)
bitboard(bs3)
bitboard(bs4)

0000100001000000000100001 0000000000000010000000000 1000010000000001000010000 0000000000100000000000000
0 | 0 0 0 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 1 1 0 1 1
    A B C D E

0 | 0 0 0 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 1 0 0
    A B C D E

0 | 1 1 0 1 1
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E

0 | 0 0 1 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E



#### Modellierung des Spielstatus

Eine Spielstellung ist vollständig beschrieben durch 
a. die Lage auf dem Brett (die 4 Bitboards)
b. die Verteilung der Karten (id0 .. id4 in einer Liste)
c. wer dran ist (der untere Spieler 0 oder der obere 1)
d. der wievielte Zug es ist. Bei 200 endet das Spiel.


```
status = [boardzahlen, cards, playerid, turn]  
```

<img src = './img/onitama01.png' width = 500>

Diese Spielstellung wäre also repräsentiert durch



In [128]:
board = startBoard()
bz = boardzahlen(board)
cards = [2, 3, 14, 8, 10]
startstate = [bz, cards, player, turn]
startstate



[(1081377, 1024, 17302032, 16384), [2, 3, 14, 8, 10], 0, 0]

----

#### goaltest

Das Spiel ist zu Ende, wenn ein Meister geschlagen wurde, oder der Meister das Anfangsfeld des gegnerischen Meisters betritt. Nach 200 Zügen ist unentschieden.

In [101]:
M1 = 1024       # Startpos Meister 1
M2 = 16384      # Startpos Meister 2
bitboard(M2)

0 | 0 0 1 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E



In [126]:
s1, m1, s2, m2 = bz 

if M2 & m1:
    print('Meister 1 hat das Heimatfeld von Meister 2 erreicht')
if M1 & m2:
    print('Meister 2 hat das Heimatfeld von Meister 1 erreicht')
if not M1:
    print('Meister 1 wurde geschlagen')
if not M2:
    print('Meister 2 wurde geschlagen')
if turn >= 200:
    print('Unentschieden')


In [131]:
def goaltest(state):
    m1 = state[0][1]
    m2 = state[0][3]
    turn = state[3]
    return M2 & m1 or M1 & m2 or not M1 or not M2 or turn >= 200

In [132]:
goaltest(startstate)

False

#### nextstates